# <center>Python Project : intercative pricing app based on Bokeh</center>
#### Ines BIDAL, Noam AFLALO

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
import bokeh
import math
import scipy.stats as stats
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.sparse import diags
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm
import numpy.linalg as lng
import ipywidgets as widgets

In [3]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure, output_file, reset_output
from bokeh.core.properties import Instance, String
from bokeh.models import ColumnDataSource, LayoutDOM
from bokeh.util.compiler import TypeScript

In [4]:
# See how the memory is working 
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2e9')
client

Client Scheduler: inproc://192.168.1.213/5184/1 Dashboard: http://192.168.1.213:51959/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


# Part I : Graphical representation using Bokeh of the accuracy of the Euler Implicit Scheme to price European options.

## A/ Pricing using Black-Scholes (Closed form formula)

In [5]:
#Black and Scholes formula for European put
def P_BS(s, K, T):
    d1 = math.log(s / (K * math.exp(-r*T))) / (sigma * math.sqrt(T)) + sigma * math.sqrt(T) / 2
    d2 = math.log(s / (K * math.exp(- r * T))) / (sigma * math.sqrt(T)) - sigma * math.sqrt(T) / 2
    Nd1 = stats.norm.cdf(- d1, loc=0, scale=1)
    Nd2 = stats.norm.cdf(- d2, loc=0, scale=1)
    P = math.exp(- r * T) * K * Nd2 - s * Nd1
    return P


#Black and Scholes formula for European call
def C_BS(s, K, T):
    d1 = math.log(s/(K* math.exp(-r*T))) / (sigma * math.sqrt(T)) + sigma * math.sqrt(T) / 2
    d2 = math.log(s/(K* math.exp(-r*T))) / (sigma * math.sqrt(T)) - sigma * math.sqrt(T) / 2
    Nd1 = stats.norm.cdf(d1, loc=0, scale=1)
    Nd2 = stats.norm.cdf(d2, loc=0, scale=1)
    C = -math.exp(r*T) * K * Nd2 + s * Nd1
    return C

## B/ Pricing using the Implicit Euler Scheme (approximation) 

In [6]:
# initialisation
S=100
K = 100
r = 0.01
T = 1
sigma = 0.2

In [7]:
# NUMERICAL PARAMETERS
N=10
I=10
S_min=0; S_max=200;
h = (S_max - S_min) / (I + 1)
dt = T / N
Sval = 80

In [8]:
# payoff function 
def u0(s, K=K) : 
    return np.maximum(0, K - s)

def u_left(t, S_min=S_min) : 
    return K * np.exp(-r * t) - S_min

def u_right(t) :
    return 0

def u_call(s, K):
    return np.maximum(0, s - K)

In [9]:
def IE(I,N, K, S0, sigma=sigma, T=T):
    
    
    # NUMERICAL PARAMETERS
    S_min = max(0, S0 - 100)
    S_max = S0 + 100
    Sval = 0.8 * S0
    dt = T / N
    h = (S_max - S_min) / I
    
    #vectors
    S = np.array([(S_min + j * h) for j in range(1,I+1)])

    tn = np.array([n * dt for n in range(N+1)])

    alpha = ((sigma ** 2)*(S ** 2)/(2 * (h ** 2)))

    beta = (r / (2 * h)) * S
    
    # q(t)

    def q(t):
        y = np.zeros((I,1))
        y[0] = ( - alpha[1] + beta[1]) * u_left(t, S_min)
        y[-1] = ( - alpha[-1] - beta[-1]) * u_right(t)
        return y

    # A
    array = list(- alpha[:- 2] - beta[:- 2])
    array.append(0)
    array = np.array(array)
    matToTridiag = np.array([- alpha[1::] + beta[1::],2 * alpha + r,array])
    offset =  [- 1,0,1]

    A = diags(matToTridiag,offset).toarray()
    
    U = u0(S, K).reshape(I, 1)
    
    for i in range(N):
        U = lng.solve(dt * A + np.eye(I), -q(tn[i + 1])*dt +U)

    
    return(U, S)

# Graphical comparison between the two methods of pricing using bokeh

In [10]:
def plotter_eu_put(I, N, K, S0, sigma=sigma, T=T) :
    V = IE(I, N, K, S0, sigma, T)
    
    opts = dict(plot_width=550, plot_height=350, min_border=0, 
            title='European Put option')
    p = figure(**opts)
    r1 = p.line(V[1], V[0].ravel(), line_width=2, legend_label='Euler Scheme')
    r2 = p.line(V[1], [u0(i, K) for i in V[1]], line_width=2, 
                color='red', legend_label='Payoff')
    r3 = p.line(V[1], [P_BS(i, K, T) for i in V[1]], line_width=1, 
               color='black', legend_label='Black & Scholes', 
                line_dash='4 4')
    p.xaxis.axis_label = 'S'
    p.yaxis.axis_label = 'Price'
    
    try :
        reset_output()
        output_notebook()
    except :
        try : 
            output_notebook()
        except :
            pass
    
    t1 = show(p, notebook_handle=True)

In [11]:
def plotter_eu_call(I, N, K, S0, sigma=sigma, T=T):
    V = IE(I, N, K, S0, sigma, T)
    call = V[0].ravel() + V[1] - K * np.exp(-r*T)
    
    opts = dict(plot_width=550, plot_height=350, min_border=0, 
            title='European Put option')
    p = figure(**opts)
    r1 = p.line(V[1], call, line_width=2, legend_label='Euler Scheme')
    r2 = p.line(V[1], [u_call(i, K) for i in V[1]], line_width=2, 
                color='red', legend_label='Payoff')
    r3 = p.line(V[1], [C_BS(i, K, T) for i in V[1]], line_width=1, 
               color='black', legend_label='Black & Scholes', 
                line_dash='4 4')
    p.xaxis.axis_label = 'S'
    p.yaxis.axis_label = 'Price'
    
    try :
        reset_output()
        output_notebook()
    except :
        try : 
            output_notebook()
        except :
            pass
    
    t1 = show(p, notebook_handle=True)

In [12]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def func(Product):
    
    if Product == 'Put' :
        w = interact(lambda I, N, K, S0, sig, T : plotter_eu_put( I, N, K, S0, sig, T), 
            I=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Spot mesh I'), 
            N=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Time Mesh N' ), 
            K=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Strike K'), 
            S0=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Spot S'), 
            sig=widgets.BoundedFloatText(value=0.2, min=0, max=1.0, step=0.1, description='Volatility:', disabled=False),
            T=widgets.BoundedFloatText(value=1, min=0, max=2.0, step=0.1, description='Maturity:', disabled=False))
    
    elif Product == 'Call' :
        w = interact(lambda I, N, K, S0, sig, T : plotter_eu_call( I, N, K, S0, sig, T), 
            I=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Spot mesh I'), 
            N=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Time Mesh N' ), 
            K=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Strike K'), 
            S0=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Spot S'), 
            sig=widgets.BoundedFloatText(value=0.2, min=0, max=1.0, step=0.1, description='Volatility:', disabled=False),
            T=widgets.BoundedFloatText(value=1, min=0, max=2.0, step=0.1, description='Maturity:', disabled=False))
        
wf = interact(func, Product=['Put', 'Call'])

interactive(children=(Dropdown(description='Product', options=('Put', 'Call'), value='Put'), Output()), _dom_c…

# Part II : Pricing of exotic options using different kinds of Monte Carlo simulation using Dask and Bokeh

In [13]:
T = 1
S0 = 100 # Initial price            
r = 0.01 # Risk free interest rate
sigma = 0.2 # Volatility
rho = -0.5 # Correlation
kappa = 2.5 # Revert rate
theta = 0.05 # Long-term volatility
xi = 0.04 # Volatility of instantaneous volatility
v0 = 0.2 # Initial instantaneous volatility
Accumulationlevel = 70 # Accumulation Level
B = 110 # Barrier Level
nb_steps = 252 #Nombre de steps
nb_path = 1000 #Nombre de paths 
S = np.arange(50,150,252)
t = np.arange(0.1,1)
surface=True

## A/ Pricing of an accumulator under constant volatility model (Black-Scholes)

In [14]:
# Nous avons rencontré un problème avec Dask : on ne peut pas assigner des parties de array exemple : arr[0] = 1 impossible a faire. 
# Une solution est de concatener des bouts de array. Cependant, on perd en rapidité, la version numpy est même préférée.
# Version Dask:
#from tqdm.autonotebook import tqdm
#def Black_Scholes_path(S0, T, r, sigma, nb_steps):
#    '''Function that generate B&S path with constant volatility sigma'''
#    S = da.from_array(np.array([S0]), chunks=(1,))
#    dt = T / nb_steps
#    dask_arrays = [S]
#    for t in range(1, nb_steps):
#        eps = np.random.standard_normal(1)   # pseudorandom numbers
# #         arr = da.from_array(np.array([dask_arrays[-1] * math.exp((r-0.5*sigma**2 ) * dt + sigma * eps * math.sqrt(dt))]), 
# #                             chunks=1)
#        arr = dask_arrays[-1] * math.exp((r-0.5*sigma**2 ) * dt + sigma * eps * math.sqrt(dt))
#        dask_arrays.append(arr)
#    return da.concatenate(dask_arrays, axis=0).compute()


In [15]:
#Version Numpy:
def Black_Scholes_path(S0, T, r, sigma, nb_steps):
    '''Function that generate B&S path with constant volatility sigma'''
    
    S = np.zeros(nb_steps)
    S[0] = S0
    dt = T / nb_steps
    
    for t in range(1, nb_steps):
        eps = np.random.standard_normal(1)   # pseudorandom numbers
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma**2 ) * dt + sigma * eps * np.sqrt(dt))
        
    return S

## B/ Pricing of an accumulator under non constant volatility model (Heston model)

In [16]:
# Nous avons rencontré un problème avec Dask : on ne peut pas assigner des parties de array exemple : arr[0] = 1 impossible a faire. 
# Une solution est de concatener des bouts de array. Cependant, on perd en rapidité, la version numpy est même préférée.
# Version Dask:
#def HeMC (S0, r, v0, rho, kappa, theta, xi, T, nb_steps):
#    '''Generate a Monte Carlo simulation for the Heston model'''
#
#    # Generate random Brownian Motion
#    R = np.array([0, 0])
#    COV = np.matrix([[1, rho], [rho, 1]])
#    W = np.random.multivariate_normal(R, COV, nb_steps)
#    W = da.from_array(W, chunks=(2, 10))
#    W_S = W[:, 0]
#    W_v = W[:, 1]
#    dt = T / nb_steps
#    # Generate paths
#    arr = np.zeros((nb_steps, 1))
#    arr[:, 0] = v0
#    v = [da.from_array(arr, chunks=1)]
#    S = [da.from_array(np.array([S0]), chunks=1)]
#
#    for t in range(1,nb_steps):
#        v_tmp = v[-1] + kappa * (theta - v[-1]) * dt + xi * np.sqrt(v[-1]) * np.sqrt(dt) * W_v[t]                  
#        v.append(v_tmp)
#
#        S_tmp = S[-1] * np.exp((r - 0.5 * v[-1][0]) * dt + np.sqrt(v[-1][0] * dt) * W_S[t])
#        S.append(S_tmp)
#
#    S = da.concatenate(S, axis=0).compute()
#    vol = da.concatenate(v, axis=1).compute()
#   
#    return S, vol

In [17]:
# Version Numpy:

def HeMC (S0, r, v0, rho, kappa, theta, xi, T, nb_steps):
    '''Generate a Monte Carlo simulation for the Heston model'''

    # Generate random Brownian Motion
    R = np.array([0, 0])
    COV = np.matrix([[1, rho], [rho, 1]])
    W = np.random.multivariate_normal(R, COV, nb_steps)
    W_S = W[:, 0]
    W_v = W[:, 1]
    dt = T / nb_steps
    # Generate paths
    v = np.zeros(nb_steps)
    v[0] = v0
    S = np.zeros(nb_steps)
    S[0] = S0
    
    
    expiry_dates = np.zeros(nb_steps)
    strikes = np.zeros(nb_steps)
    strikes[0] = S0
    vol = np.zeros((nb_steps,nb_steps))
    vol[:, 0] = v0
    
    for t in range(1,nb_steps):
        expiry_dates[t] = t * dt
        v[t] = v[t-1] + kappa * (theta - v[t-1]) * dt + xi * np.sqrt(v[t-1]) * np.sqrt(dt) * W_v[t]
        vol[:, t] = v[t]
        S[t] = S[t-1] * np.exp((r - 0.5 * v[t-1]) * dt + np.sqrt(v[t-1] * dt) * W_S[t])
        strikes = S[t]
    return S, vol


In [18]:
def Accumulator(S0, kappa, theta, xi, Accumulationlevel, 
                sigma, B, Type, Model, T=T,
                nb_path=nb_path, nb_steps=nb_steps, r=r):  ## ca calcule le prix d'un accu, on precise si on veut priceer avec barrier continue ou discret + si on veut faire avec vol constante ou stochastique. 
    
    currentPayoff = np.zeros(nb_path)
    
    if Type == "Continuous":
            shift=np.exp(0.5826 * sigma * np.sqrt(T/nb_steps))
            B = B / shift
            
    for i in range(nb_path):
        if Model == "BS":
            S = Black_Scholes_path(S0, T, r, sigma, nb_steps)
        else: 
            S= HeMC (S0, r, v0, rho, kappa, theta, xi, T, nb_steps)[0]
            if (i == 0 and surface):
                plotter_heston(S0, kappa, theta, xi, v0, rho, nb_steps, 
                              r, T)  # Si on fait un pricing avec vol stochastique alors on affiche aussi une surface de volatilite utilise. 
        for t in range(1, nb_steps):
            if S[t] > B:
                break
            elif (S[t] < B) & (S[t] > Accumulationlevel):
                currentPayoff[i] = currentPayoff[i] + (S[t]- Accumulationlevel)
                
            else: 
                currentPayoff[i] = currentPayoff[i] + 2 * (S[t]- Accumulationlevel)
    print('The price for this configuration is : '+str(np.exp(-r * T) * np.mean(currentPayoff)))
    return np.exp(-r * T) * np.mean(currentPayoff)  

In [19]:
# Code Java Script
TS_CODE = """

import {LayoutDOM, LayoutDOMView} from "models/layouts/layout_dom"
import {ColumnDataSource} from "models/sources/column_data_source"
import {LayoutItem} from "core/layout"
import * as p from "core/properties"

declare namespace vis {
  class Graph3d {
    constructor(el: HTMLElement, data: object, OPTIONS: object)
    setData(data: vis.DataSet): void
  }

  class DataSet {
    add(data: unknown): void
  }
}

const OPTIONS = {
  width: '600px',
  height: '600px',
  style: 'surface',
  showPerspective: true,
  showGrid: true,
  keepAspectRatio: true,
  verticalRatio: 1.0,
  legendLabel: 'stuff',
  cameraPosition: {
    horizontal: -0.35,
    vertical: 0.22,
    distance: 1.8,
  },
}

export class Surface3dView extends LayoutDOMView {
  model: Surface3d

  private _graph: vis.Graph3d

  initialize(): void {
    super.initialize()

    const url = "https://cdnjs.cloudflare.com/ajax/libs/vis/4.16.1/vis.min.js"
    const script = document.createElement("script")
    script.onload = () => this._init()
    script.async = false
    script.src = url
    document.head.appendChild(script)
  }

  private _init(): void {

    this._graph = new vis.Graph3d(this.el, this.get_data(), OPTIONS)

    this.connect(this.model.data_source.change, () => {
      this._graph.setData(this.get_data())
    })
  }

  get_data(): vis.DataSet {
    const data = new vis.DataSet()
    const source = this.model.data_source
    for (let i = 0; i < source.get_length()!; i++) {
      data.add({
        x: source.data[this.model.x][i],
        y: source.data[this.model.y][i],
        z: source.data[this.model.z][i],
      })
    }
    return data
  }

  get child_models(): LayoutDOM[] {
    return []
  }

  _update_layout(): void {
    this.layout = new LayoutItem()
    this.layout.set_sizing(this.box_sizing())
  }
}

export namespace Surface3d {
  export type Attrs = p.AttrsOf<Props>

  export type Props = LayoutDOM.Props & {
    x: p.Property<string>
    y: p.Property<string>
    z: p.Property<string>
    data_source: p.Property<ColumnDataSource>
  }
}

export interface Surface3d extends Surface3d.Attrs {}

export class Surface3d extends LayoutDOM {
  properties: Surface3d.Props
  __view_type__: Surface3dView

  constructor(attrs?: Partial<Surface3d.Attrs>) {
    super(attrs)
  }

  static __name__ = "Surface3d"

  static init_Surface3d() {
    this.prototype.default_view = Surface3dView

    this.define<Surface3d.Props>({
      x:            [ p.String   ],
      y:            [ p.String   ],
      z:            [ p.String   ],
      data_source:  [ p.Instance ],
    })
  }
}
"""

class Surface3d(LayoutDOM):
    __implementation__ = TypeScript(TS_CODE)
    data_source = Instance(ColumnDataSource)

    x = String

    y = String

    z = String

In [20]:
def plotter_heston(S0, kappa, theta, xi, 
                   v0=0.2, rho=-0.2, nb_steps=252, r=0.1, T=1):
    vol = HeMC (S0, r, v0, rho, kappa, theta, xi, T, nb_steps)[1]
    ny, nx=vol.shape
    x = np.linspace(T, 0, nx)
    y = np.linspace(0, 2*S0, ny)
    xv, yv = np.meshgrid(x, y)
    xv = xv.ravel() * 2e6
    yv = yv.ravel() * 1e4
    zv = vol.ravel() * 1e4
    data = dict(x=xv, y=yv, z=zv)
    source = ColumnDataSource(data=data)
    surface = Surface3d(x="x", y="y", z="z", data_source=source, width=6000, height=6000)
    reset_output()
    output_file('foo.html')
    show(surface)

In [21]:
w = interact(lambda S, k, theta, xi, Acc, sig, B, Type, Model, T, n_p, n_s : Accumulator(S, k, theta, xi, Acc, sig, B, Type, Model, T, n_p, n_s), 
             S=widgets.IntSlider(min=0, max=200, step=1, value=100, description='Spot Price:'),
             k=widgets.FloatText(value=0.3, description='Revert Rate:', disabled=False),
             theta=widgets.FloatText(value=0.2, description='Long Term Vol:', disabled=False),
             xi=widgets.FloatText(value=0.2, description='Vol of vol:', disabled=False), 
             Acc=widgets.IntSlider(min=0, max=200, step=10, value=70, description='Accu Level:'), 
             sig=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.1, description='BS Vol:'),
             B=widgets.IntSlider(min=0, max=200, step=10, value=110, description='Barrier:'),
             Type=['Continuous', 'Discret'], 
             Model=['BS', 'Heston'], 
             T=widgets.BoundedFloatText(value=1, min=0, max=2.0, step=0.1, description='T:', disabled=False),
             n_p=widgets.IntText(value=1000, description='Number path:', disabled=False),
             n_s=widgets.IntText(value=252, description='Number Steps:', disabled=False))


interactive(children=(IntSlider(value=100, description='Spot Price:', max=200), FloatText(value=0.3, descripti…

# Conclusion: API summarizing the whole project 

In [22]:
def func(Product):
    if Product == 'Put' :
        w = interact(lambda I, N, K, S0, sig, T : plotter_eu_put( I, N, K, S0, sig, T), 
            I=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Spot mesh I'), 
            N=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Time Mesh N' ), 
            K=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Strike K'), 
            S0=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Spot S'), 
            sig=widgets.BoundedFloatText(value=0.2, min=0, max=1.0, step=0.1, description='Volatility:', disabled=False),
            T=widgets.BoundedFloatText(value=1, min=0, max=2.0, step=0.1, description='Maturity:', disabled=False))
    
    elif Product == 'Call' :
        w = interact(lambda I, N, K, S0, sig, T : plotter_eu_call( I, N, K, S0, sig, T), 
            I=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Spot mesh I'), 
            N=widgets.IntSlider(min=10, max=30, step=1, value=20, description='Time Mesh N' ), 
            K=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Strike K'), 
            S0=widgets.IntSlider(min=10, max=200, step=10, value=100, description='Spot S'), 
            sig=widgets.BoundedFloatText(value=0.2, min=0, max=1.0, step=0.1, description='Volatility:', disabled=False),
            T=widgets.BoundedFloatText(value=1, min=0, max=2.0, step=0.1, description='Maturity:', disabled=False))
        
    elif Product == 'Accumulator' :
        w = interact(lambda S, k, theta, xi, Acc, sig, B, Type, Model, T, n_p, n_s : Accumulator(S, k, theta, xi, Acc, sig, B, Type, Model, T, n_p, n_s), 
             S=widgets.IntSlider(min=0, max=200, step=1, value=100, description='Spot Price:'),
             k=widgets.FloatText(value=0.3, description='Revert Rate:', disabled=False),
             theta=widgets.FloatText(value=0.2, description='Long Term Vol:', disabled=False),
             xi=widgets.FloatText(value=0.2, description='Vol of vol:', disabled=False), 
             Acc=widgets.IntSlider(min=0, max=200, step=10, value=70, description='Accu Level:'), 
             sig=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.1, description='BS Vol:'),
             B=widgets.IntSlider(min=0, max=200, step=10, value=110, description='Barrier:'),
             Type=['Continuous', 'Discret'], 
             Model=['BS', 'Heston'], 
             T=widgets.BoundedFloatText(value=1, min=0, max=2.0, step=0.1, description='T:', disabled=False),
             n_p=widgets.IntText(value=1000, description='Number path:', disabled=False),
             n_s=widgets.IntText(value=252, description='Number Steps:', disabled=False))


wf = interact(func, Product=['Put', 'Call', 'Accumulator'])

interactive(children=(Dropdown(description='Product', options=('Put', 'Call', 'Accumulator'), value='Put'), Ou…

# Testing of the code using the module ipytest

In [23]:
import ipytest
ipytest.autoconfig()

In [24]:
%%run_pytest[clean]

def test_vanilla_options():
    assert np.round(P_BS(100, 100, 1),0) == np.round(C_BS(100, 100, 1),0) #ATM Call et Put options have the same price.
    assert np.abs((C_BS(100, 150, 1)- P_BS(100, 150, 1) - (100 - 150))/(100 - 150))<0.03 # Verification of the put call parity.

def test_comparison_BS_Euler():
    #Verifications that the Euler implicit scheme give approximately the same results as the BS closed form formula.
    assert np.round(np.abs(IE(20,20, 100, 100, sigma=sigma, T=T)[0][9]- P_BS(100, 100, 1)),0) == 0
    Smax= 2 * 100
    assert np.round(np.abs(IE(20,20, 100, 100, sigma=sigma, T=T)[0][19]-P_BS(Smax, 100, 1)),0) == 0
    assert np.round(np.abs((IE(20,20, 100, 100, sigma=sigma, T=T)[0][0]-P_BS(0.0001, 100, 1))/P_BS(0.0001, 100, 1)),0)<0.01


..                                                                       [100%]
2 passed in 0.05s


The folowwing tests check the following:

1) All parameters being equal, an accumulator monitored discretely should be more expansive than an accumulator monitored continuously. Indeed, the probability that the barrier is reached is higher when it is monitored continuously. When the barrier level is breached then the consumer does not accumulate anymore while its objective is to buy at premium as much as he can (except in the very uncommon situation where the consumer does not want to buy at premium more than a certain quantity). Hence, it is not in his interest that the knock-out event happens.


2) All parameters being equal, when the accumulation level gets closer to the barrier level then the price of the structure should decreases. Indeed, if the price at which the consumer is going to buy the underlying increases then the value of the accumulator must decrease.

In [25]:
%%run_pytest[clean]


def test_accumulator_BS_barrier():
    assert Accumulator(S0, kappa, theta, xi, Accumulationlevel, 
                sigma, B, 'Discretely', 'BS', T=T,
                nb_path=nb_path, nb_steps=nb_steps, r=r) > Accumulator(S0, kappa, theta, xi, Accumulationlevel, 
                sigma, B, 'Continuous', 'BS', T=T,
                nb_path=nb_path, nb_steps=nb_steps, r=r)

def test_accumulator_BS_moneyness():
    assert Accumulator(S0, kappa, theta, xi, 70, 
                sigma, B, 'Discretely', 'BS', T=T,
                nb_path=nb_path, nb_steps=nb_steps, r=r) > Accumulator(S0, kappa, theta, xi, 80, 
                sigma, B, 'Continuous', 'BS', T=T,
                nb_path=nb_path, nb_steps=nb_steps, r=r)   


..                                                                       [100%]
2 passed in 13.90s
